## Load Dependencies

In [1]:
# load dependencies
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

## Load Dataset

In [2]:
# load the dataset
url = "iris.data"

# column names to use
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

# read the dataset from URL
dataset = pd.read_csv(url, names=names) 

# check first few rows of data
dataset.head()

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Split Data and Standardize Features

In [3]:
# seperate the independent and dependent features
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values 

# Split dataset into random training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) 

# feature standardization
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 

## Train Classifier

In [4]:
# training a KNN classifier
clf_model = KNeighborsClassifier(n_neighbors=5)
clf_model.fit(X_train, y_train) 

KNeighborsClassifier()

## Check Classifier Performance

In [5]:
# make prediction on the testing data
y_predict = clf_model.predict(X_test)

# check results
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict)) 

[[11  0  0]
 [ 0  7  1]
 [ 0  3  8]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       0.70      0.88      0.78         8
 Iris-virginica       0.89      0.73      0.80        11

       accuracy                           0.87        30
      macro avg       0.86      0.87      0.86        30
   weighted avg       0.88      0.87      0.87        30



## Save Model with Neptune Model Registry

In [6]:
# import neptune
# model = neptune.init_model(
#     name="Prediction model",
#     key="MOD", 
#     project="gouravbais08/iris-classification", 
#     api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1OWMxNDZlMS0zYWFlLTQ1YTMtYTgyZC00ZDg2YTQ2OWYxMzMifQ==", # your credentials
# )

In [7]:
import neptune
model = neptune.init_model(
    name="Prediction model",
    key="IRMOD", 
    project="gouravbais08/iris-classification", 
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1OWMxNDZlMS0zYWFlLTQ1YTMtYTgyZC00ZDg2YTQ2OWYxMzMifQ==", # your credentials
)

https://app.neptune.ai/gouravbais08/iris-classification/m/IR-IRMOD


In [8]:
# assign the model metadata to model object 
model_info = {"size_limit": 7.09, "size_units": "KB"}
model["model"] = model_info

In [9]:
# upload the model to neptune model registry
model["model/signature"].upload("iris_classifier_model.pkl")

In [10]:
# track dataset version
model["data/train_and_test"].track_files("iris.data")

In [11]:
model.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.
All 5 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/gouravbais08/iris-classification/m/IR-IRMOD/metadata


## Version the Model

In [12]:
# initialize the ModelVersion 
import neptune
model_version = neptune.init_model_version(
    model="IR-IRMOD",
    project="gouravbais08/iris-classification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1OWMxNDZlMS0zYWFlLTQ1YTMtYTgyZC00ZDg2YTQ2OWYxMzMifQ==", # your credentials
)

https://app.neptune.ai/gouravbais08/iris-classification/m/IR-IRMOD/v/IR-IRMOD-1


In [13]:
# model parameters
parameters = {
    "algorithm": clf_model.get_params()['algorithm'],
    "max_iter": clf_model.get_params()['leaf_size'],
    "solver": clf_model.get_params()['metric'],
    "metric_params": clf_model.get_params()['metric_params'],
    "n_jobs": clf_model.get_params()['n_jobs'],
    "n_neighbors": clf_model.get_params()['n_neighbors'],
    "p": clf_model.get_params()['p'],
    "weights": clf_model.get_params()['weights'],
}

# log model parameters and other metadata
model_version["model/parameters"] = parameters
model_version["data/dataset"].track_files("iris.data")
model_version["validation/acc"] = 0.93

/var/folders/0k/ftvl7p0j58v4p29wsgrkgnvr0000gn/T/ipykernel_6330/3774705092.py:14: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'NoneType'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  model_version["model/parameters"] = parameters


In [14]:
model_version["model/binary"].upload("iris_classifier_model.pkl")
model_version.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 9 operations to synchronize with Neptune. Do not kill this process.
All 9 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/gouravbais08/iris-classification/m/IR-IRMOD/v/IR-IRMOD-1/metadata


## Querying the Model and Metadata

In [55]:
import neptune
import pickle

# specify model version id
version_id = 'IR-IRMOD-1' 

# initialize run 
model_version = neptune.init_model_version(
    with_id=version_id,
    project="gouravbais08/iris-classification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1OWMxNDZlMS0zYWFlLTQ1YTMtYTgyZC00ZDg2YTQ2OWYxMzMifQ==", # your credentials
)

https://app.neptune.ai/gouravbais08/iris-classification/m/IR-IRMOD/v/IR-IRMOD-1


In [56]:
# save model
if model_version.exists("model/binary"):
    model_version["model/binary"].download(f"dataset/{version_id}_model.pkl")

In [59]:
# load model from pickle file
with open(f"dataset/{version_id}_model.pkl", 'rb') as file:  
    clf_model_2 = pickle.load(file)

In [60]:
# evaluate model 
y_predict = clf_model_2.predict(X_test)

# check results
print(classification_report(y_test, y_predict)) 

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       0.80      1.00      0.89         8
 Iris-virginica       1.00      0.82      0.90        11

       accuracy                           0.93        30
      macro avg       0.93      0.94      0.93        30
   weighted avg       0.95      0.93      0.93        30



In [61]:
# check metadata 
model_version["model/parameters"].fetch()

{'algorithm': 'auto',
 'max_iter': 30,
 'n_neighbors': 5,
 'p': 2,
 'solver': 'minkowski',
 'weights': 'uniform'}

In [62]:
# download dataset
model_version["data/dataset"].download(f"dataset/{version_id}_data.data")

In [63]:
# load the dataset 
import pandas as pd 

# load dataset 
iris = pd.read_csv("dataset/IR-IRMOD-1_data.data/iris.data")

# check first few rows
iris.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [40]:
# close session 
model.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/gouravbais08/iris-classification/m/IR-IRMOD/metadata
